# Expose Iceberg - Uniform


[How to Read Unity Catalog Tables in Snowflake, in 4 Easy Steps](https://www.databricks.com/blog/read-unity-catalog-tables-in-snowflake)

There are 4 steps to creating a REST catalog integration in Snowflake:

- Enable UniForm on a Delta Lake table in Databricks to make it accessible through the Iceberg REST Catalog
- Register Unity Catalog in Snowflake as your catalog
- Register an S3 Bucket in Snowflake so it recognizes the source data
- Create an Iceberg table in Snowflake so you can query your data

1. With UC as delta catalog 
2. With UC + iceberg-rest endpoint

## With Unity Catalog 

- UC required
- Enable column mapping
- minReaderVersion >= 2 and minWriterVersion >= 7. 
- DBR > 14.3LTS
- Disable DV

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
table_path = dbutils.widgets.get("path")

In [0]:
%sql
use catalog ${catalog};
use ${schema}

In [0]:
%sql 
describe detail foo

In [0]:
%sql
describe  schema ${catalog}.${schema}

In [0]:
%sql
if exists drop table uniform_demo_table

In [0]:
dbutils.fs.mkdirs('dbfs:/uniform_demo')

In [0]:
%sql
create or replace table  uniform_demo_table (id INT, name STRING)
USING DELTA
tblproperties (
  'delta.columnMapping.mode' = 'name',
  'delta.enableIcebergCompatV2' = 'true',
  'delta.universalFormat.enabledFormats' = 'iceberg'
)

LOCATION 'dbfs:/uniform_demo'
;

In [0]:
%sql
DESCRIBE EXTENDED uniform_demo_table

In [0]:
for i in range(1, 20):
    spark.sql(f"INSERT INTO uniform_demo_table VALUES ({i}, 'Name_{i}')")


In [0]:
%sql
describe detail uniform_demo_table

In [0]:
%python
display(dbutils.fs.ls(table_path))


In [0]:
%sql
describe extended uniform_demo_table

In [0]:
display(dbutils.fs.ls(f"{table_path}/metadata/"))

In [0]:
print(display(dbutils.fs.head(f"{table_path}/metadata/00009-212b965d-5d15-45c0-9b1e-b451222aae6a.metadata.json")))

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "False")

In [0]:
%sql
VACUUM uniform_demo_table RETAIN 0 HOURS

Iceberg metadata generation is done asynchronously. 

You can manually trigger Iceberg metadata conversion


In [0]:
%sql
MSCK REPAIR TABLE uniform_demo_table SYNC METADATA

## Hive table

In [0]:

display(spark.sql(f"describe extended delta.`{table_path}`"))